In [9]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,  # before summerized
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Jimin, I live in South Korea", "Wow that is so cool!")

In [10]:
get_history()

{'history': [HumanMessage(content="Hi I'm Jimin, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [11]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [12]:
get_history()

{'history': [HumanMessage(content="Hi I'm Jimin, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

In [13]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [14]:
get_history()

{'history': [HumanMessage(content="Hi I'm Jimin, I live in South Korea"),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [18]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [19]:
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Jimin from South Korea.'),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Brazil from Argentina?'),
  AIMessage(content="I don't know! Super far!")]}

In [ ]:
# gpt-4o-mini-2024-07-18 and etc.

NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model cl100k_base.See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.